In [1]:
!which python
import csv
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import sklearn
from scipy.stats import spearmanr as scpiy_spearmanr
import time
from utils import *
from matplotlib import pyplot as plt
import wandb
import uuid
import os
from models import *
import numerapi
import pathlib
import torchsort
import tensorflow as tf
from adahessian import AdaHessian
import json
import uuid
from tqdm import tqdm 
from notion.client import NotionClient
from notion.block import *
import gc
torch.autograd.set_detect_anomaly(True)
with open("secrets.json") as s:
    secrets = json.load(s)
NAPI = numerapi.NumerAPI(verbosity="info",public_id=secrets["NUMERAPI_PUBLIC_ID"],secret_key=secrets["NUMERAPI_SECRET_KEY"])
notion_client = NotionClient(token_v2=secrets["NOTION_TOKENV2"])
os.environ['WANDB_NOTEBOOK_NAME'] = 'main.ipynb'
os.environ['WANDB_BASE_URL'] = "https://api.wandb.ai"

/Users/ryandu_1/miniforge3/envs/mlenv/bin/python


In [2]:
# refresh_numerai_data()
training_data, tournament_data, feature_names = get_dataset()
training_data = NumeraiTrainData(training_data,feature_names)
train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=158, shuffle=True)
tournament_data = NumeraiTournamentData(tournament_data,feature_names)
tournament_loader = torch.utils.data.DataLoader(tournament_data, batch_size=158, shuffle=True)

numerai train init took 3.7951300144195557
numerai tourn init took 17.799970149993896


## Main Initializing, Training, and Val + Reuslts

In [3]:
## DO feature engineering 
# important_features = getImportantFeatures(vanilla_model,training_data[:20000],feature_names)
# with open(f"results/round{NAPI.get_current_round()}/importantFeatures1.json",'x') as f:
    # f.write(json.dumps(important_features))
## get already DONE feature engineering
with open(f"results/importantFeatures1.json",) as f:
    important_features = json.load(f)
smaller_training_data = NumeraiTrainData(training_data.training_data,important_features)
smaller_train_dataloader = torch.utils.data.DataLoader(smaller_training_data, batch_size=158, shuffle=True)

numerai train init took 1.0027439594268799


In [4]:
#Vanilla NNE-SL
vanilla_model = VanillaNNEnsembleSingleLayer(train_dataloader=train_dataloader,citizens=32,citizen_layers=8,citizen_neurons=128,use_wandb=False,feature_names=important_features)


does your model already have a notion data point? (y/n)
n
Enter a title for this model: 
Vanilla NN Ensemble Smaller Feature Set AdaHessian
what loss function will you be relying on mostly? 
mse
MODEL ID: 91f6c652-f97a-4e0e-bc15-59b83880e4ad


/Users/ryandu_1/miniforge3/envs/mlenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
vanilla_model.train_(epochs=20, loss_fn=F.mse_loss,optimizer_fn=AdaHessian,lb_create_graph=True,init_lr=0.1,gamma=0.995)

spearman: 0.042313410198553905, mse: 0.04714767262339592:   1%|          | 38/3176 [01:01<1:23:11,  1.59s/it]

KeyboardInterrupt: 

In [7]:
torch.save(vanilla_model.state_dict(),f'results/models/vanilla-v2-lessfeatures/{vanilla_model.mid}-20epoch.ckpt.pt')

In [8]:
!mkdir results/submissions/round{NAPI.get_current_round()}
!mkdir results/submissions/round{NAPI.get_current_round()}/submissions/
full_tourn_results(chocolate_model,important_features,tournament_data.tournament_data,pth=f"results/submissions/round{NAPI.get_current_round()}/submissions/{vanilla_model.notion_model_page.title}-1.csv")


mkdir: results/submissions/round266: File exists
mkdir: results/submissions/round266/submissions/: File exists
Generating predictions...



  0%|          | 0/344 [00:00<?, ?it/s]

len of pred1: 344



 45%|████▌     | 156/344 [01:13<01:24,  2.23it/s]


 90%|█████████ | 310/344 [02:22<00:14,  2.32it/s]


...

On validation the correlation has mean -0.0009328847659831307 and std 0.012026097720684931


2021-05-30 11:37:04,802 INFO numexpr.utils: NumExpr defaulting to 8 threads.


On validation the average per-era payout is -0.0009328847659831307
Validation Sharpe: -0.07757169346616613
checking max drawdown...
max drawdown: -0.07146840796699158
Max Feature Exposure: 0.03708558073154568
Calculating feature neutral mean...


/Users/ryandu_1/miniforge3/envs/mlenv/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/ryandu_1/miniforge3/envs/mlenv/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Feature Neutral Mean is 0.00034775159095870204


mkdir: results/submissions/round266/submissions/: File exists


## LETS TRY TENSORFLOW

In [36]:
#era to int
train_era_indexes = training_data["era"].unique()
training_data["era"]=training_data["era"].map(lambda a: np.where(train_era_indexes==a)[0][0]+1)
tourn_era_indexes = tournament_data["era"].unique()
tournament_data["era"]=tournament_data["era"].map(lambda a: np.where(tourn_era_indexes==a)[0][0]+1)

x_train = training_data[feature_names].to_numpy()
target = training_data["target"]
y_train = target.to_numpy()
# batch=158
# x_train = x_train.reshape(-1,batch, 310)
# y_train = y_train.reshape(-1,batch)

In [46]:
tf_model_test = tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu",kernel_regularizer="l2"), 
    tf.keras.layers.Dense(1,activation="sigmoid"),
])
tf_model_test.compile(optimizer="adam",loss="mse")

In [ ]:
tf_model_test.fit(x_train,y_train,batch_size=158)

In [44]:
get_batch_size(y_train.shape[0])

3176